Exploring the participants and the amount of data we have in different databases in Mongodb

In [1]:
import pandas as pd
import os
import sys
import numpy as np
np.seterr(divide = 'ignore') 
import pytz
import time
from datetime import datetime
from functools import reduce
from tqdm import tqdm
tqdm.pandas()

In [9]:
dataPath1 = "/csv/backup_frigg1"
dataPath2 = "/csv/backup"
featurePath = "/csv/features"

backup_frigg1 lockstate data

Lock_state data

In [10]:
dataFilename = "Lock_state.csv"
screenstate = pd.read_csv(os.path.join(dataPath1, dataFilename), sep="|", header=None)
header_list = ["id", "participant", "attribute", "lockstate", "timestamp", "uploadtimestamp"]
screenstate.columns = header_list

#change time to Halifax time
screenstate["timestamp"] = pd.to_datetime(screenstate["timestamp"], utc=True)
screenstate["timestamp"] = pd.to_datetime(screenstate["timestamp"]).dt.tz_convert(tz='America/Halifax')
screenstate["timestamp"] = pd.to_datetime(screenstate["timestamp"], utc=False)

#add new columns to help extract features
screenstate["date"] = screenstate["timestamp"].dt.date
screenstate["time"] = screenstate["timestamp"].dt.strftime('%H:%M:%S')

# keep only Covid participants
screenstate = screenstate[screenstate["participant"].str.contains("PROSITC")]

# print(screenstate.info())


dfa1 = screenstate.groupby("participant")["date"].nunique().reset_index(name="noOfDates")
dfa2 = screenstate.groupby("participant")["date"].agg(["min", "max", "count"]).reset_index().rename(columns={'min': 'minDate', 'max': 'maxDate', 'count':'totalCount'})
dfa3 = pd.merge(dfa1, dfa2, on="participant", how="inner")

# participants with more than 14days of data
dfa3 = dfa3[dfa3["noOfDates"]>=14].reset_index(drop=True)

# dfa3

In [11]:
# save this file

backup lockstate data

In [12]:
dataFilename = "Lock_state.csv"
screenstate = pd.read_csv(os.path.join(dataPath2, dataFilename), sep="|", header=None)
header_list = ["id", "participant", "attribute", "lockstate", "timestamp", "uploadtimestamp"]
screenstate.columns = header_list

#change time to Halifax time
screenstate["timestamp"] = pd.to_datetime(screenstate["timestamp"], utc=True)
screenstate["timestamp"] = pd.to_datetime(screenstate["timestamp"]).dt.tz_convert(tz='America/Halifax')
screenstate["timestamp"] = pd.to_datetime(screenstate["timestamp"], utc=False)

#add new columns to help extract features
screenstate["date"] = screenstate["timestamp"].dt.date
screenstate["time"] = screenstate["timestamp"].dt.strftime('%H:%M:%S')

# keep only Covid participants
screenstate = screenstate[screenstate["participant"].str.contains("PROSITC")]

# print(screenstate.info())


dfb1 = screenstate.groupby("participant")["date"].nunique().reset_index(name="noOfDates")
dfb2 = screenstate.groupby("participant")["date"].agg(["min", "max", "count"]).reset_index().rename(columns={'min': 'minDate', 'max': 'maxDate', 'count':'totalCount'})
dfb3 = pd.merge(dfb1, dfb2, on="participant", how="inner")

# participants with more than 14days of data
dfb3 = dfb3[dfb3["noOfDates"]>=14].reset_index(drop=True)

# dfb3

In [13]:
# only 1 participant data exists in both backupfrigg1 and backupfrigg2
pd.merge(dfa3, dfb3, on="participant", how="inner")

,participant,noOfDates_x,minDate_x,maxDate_x,totalCount_x,noOfDates_y,minDate_y,maxDate_y,totalCount_y
0,PROSITC1705,65,2021-02-25,2021-05-08,3165,16,2021-05-14,2021-05-31,60


In [14]:
# participants with more that 14days of data

print(f"participants in backup_frigg1:{dfa3.shape[0]},  participants in backup:{dfb3.shape[0]}, total Participants:{dfa3.shape[0]+dfb3.shape[0]}")

df3 = pd.concat([dfa3, dfb3], axis=0)
df3

participants in backup_frigg1:438,  participants in backup:65, total Participants:503


,participant,noOfDates,minDate,maxDate,totalCount
0,PROSITC000154,23,2020-07-19,2020-08-13,1043
1,PROSITC000181,32,2020-07-14,2020-08-17,3202
2,PROSITC000185,24,2020-07-14,2020-08-07,3918
3,PROSITC000189,31,2020-07-14,2020-08-15,13716
4,PROSITC000190,18,2020-07-16,2020-08-05,789
...,...,...,...,...,...
60,PROSITC3201,14,2022-02-15,2022-02-28,2032
61,PROSITC3221,14,2022-02-15,2022-02-28,880
62,PROSITC3226,14,2022-02-15,2022-02-28,1247
63,PROSITC3233,14,2022-02-15,2022-02-28,1419


In [15]:
# list of participants from backup and backup_frigg1 who have more that 14days of data

df3.participant.tolist()

['PROSITC000154',
 'PROSITC000181',
 'PROSITC000185',
 'PROSITC000189',
 'PROSITC000190',
 'PROSITC000191',
 'PROSITC000195',
 'PROSITC000219',
 'PROSITC000226',
 'PROSITC000234',
 'PROSITC0003',
 'PROSITC0007',
 'PROSITC0008',
 'PROSITC0010',
 'PROSITC0016',
 'PROSITC0019',
 'PROSITC0022',
 'PROSITC00239',
 'PROSITC0030',
 'PROSITC0037',
 'PROSITC0041',
 'PROSITC0043',
 'PROSITC0048',
 'PROSITC0052',
 'PROSITC0055',
 'PROSITC0057',
 'PROSITC0059',
 'PROSITC0061',
 'PROSITC0063',
 'PROSITC0065',
 'PROSITC0067',
 'PROSITC0072',
 'PROSITC0074',
 'PROSITC0078',
 'PROSITC0083',
 'PROSITC0086',
 'PROSITC0087',
 'PROSITC0088',
 'PROSITC0090',
 'PROSITC0094',
 'PROSITC0096',
 'PROSITC0100',
 'PROSITC0101',
 'PROSITC0102',
 'PROSITC0108',
 'PROSITC0110',
 'PROSITC0117',
 'PROSITC0121',
 'PROSITC0122',
 'PROSITC0124',
 'PROSITC0138',
 'PROSITC0146',
 'PROSITC0155',
 'PROSITC0170',
 'PROSITC0171',
 'PROSITC0204',
 'PROSITC0208',
 'PROSITC0215',
 'PROSITC0222',
 'PROSITC0244',
 'PROSITC0247',
 'P

In [3]:
# Personality scores
# scores = pd.read_csv("/csv/personality.csv")
# scores = scores[scores.record_id.str.contains("PROSITC")]

# labels
# file has personality score and diagnosis labels
labels = pd.read_csv("/csv/labels")
labels = labels[labels["participant"].str.contains("PROSITC")]

In [4]:
labels = labels[~labels["dass"].isnull()]

In [7]:
labels.participant.tolist()

['PROSITC001',
 'PROSITC002',
 'PROSITC003',
 'PROSITC004',
 'PROSITC005',
 'PROSITC006',
 'PROSITC007',
 'PROSITC008',
 'PROSITC010',
 'PROSITC011',
 'PROSITC012',
 'PROSITC013',
 'PROSITC014',
 'PROSITC016',
 'PROSITC017',
 'PROSITC019',
 'PROSITC021',
 'PROSITC022',
 'PROSITC023',
 'PROSITC026',
 'PROSITC028',
 'PROSITC029',
 'PROSITC030',
 'PROSITC031',
 'PROSITC032',
 'PROSITC033',
 'PROSITC035',
 'PROSITC036',
 'PROSITC037',
 'PROSITC038',
 'PROSITC039',
 'PROSITC040',
 'PROSITC041',
 'PROSITC042',
 'PROSITC043',
 'PROSITC045',
 'PROSITC046',
 'PROSITC047',
 'PROSITC048',
 'PROSITC050',
 'PROSITC051',
 'PROSITC052',
 'PROSITC053',
 'PROSITC055',
 'PROSITC056',
 'PROSITC057',
 'PROSITC058',
 'PROSITC059',
 'PROSITC060',
 'PROSITC061',
 'PROSITC062',
 'PROSITC063',
 'PROSITC064',
 'PROSITC065',
 'PROSITC066',
 'PROSITC067',
 'PROSITC070',
 'PROSITC072',
 'PROSITC073',
 'PROSITC074',
 'PROSITC075',
 'PROSITC078',
 'PROSITC079',
 'PROSITC081',
 'PROSITC082',
 'PROSITC083',
 'PROSITC0

In [8]:
df3.participant.unique()

NameError: name 'df3' is not defined

In [41]:
df_temp = pd.merge(df3, labels, on="participant", how="inner")
df_temp

,participant,noOfDates,minDate,maxDate,totalCount,con,agree,extra,neuro,open,age,gender,phone,dass,current_diagnosis
0,PROSITC1005,31,2020-12-25,2021-01-29,2944,5.0,8.0,5.0,9.0,6.0,19.0,1.0,1.0,1.0,0.0
1,PROSITC1014,29,2020-12-22,2021-01-19,3717,7.0,8.0,4.0,9.0,8.0,19.0,0.0,1.0,1.0,0.0
2,PROSITC1018,29,2021-01-04,2021-02-01,3470,9.0,9.0,6.0,5.0,9.0,25.0,0.0,1.0,3.0,0.0
3,PROSITC1025,31,2021-01-10,2021-02-09,4241,6.0,8.0,7.0,10.0,9.0,19.0,0.0,1.0,3.0,0.0
4,PROSITC1026,34,2021-01-10,2021-02-13,2445,6.0,9.0,4.0,7.0,4.0,23.0,1.0,1.0,2.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226,PROSITC3114,26,2022-02-03,2022-02-28,921,NaN,NaN,NaN,NaN,NaN,45.0,0.0,1.0,1.0,0.0
227,PROSITC3117,27,2022-02-02,2022-02-28,2353,NaN,NaN,NaN,NaN,NaN,44.0,0.0,1.0,1.0,1.0
228,PROSITC3121,17,2022-02-02,2022-02-23,994,NaN,NaN,NaN,NaN,NaN,24.0,0.0,1.0,2.0,1.0
229,PROSITC3134,17,2022-02-03,2022-02-27,954,NaN,NaN,NaN,NaN,NaN,18.0,0.0,1.0,2.0,0.0


In [42]:
# out of 503 participants only 231 participants have labels

df3.shape, df_temp.shape

((503, 5), (231, 15))

In [44]:
df_temp.participant.tolist()

['PROSITC1005',
 'PROSITC1014',
 'PROSITC1018',
 'PROSITC1025',
 'PROSITC1026',
 'PROSITC1027',
 'PROSITC1029',
 'PROSITC1030',
 'PROSITC1031',
 'PROSITC1033',
 'PROSITC1034',
 'PROSITC1036',
 'PROSITC1037',
 'PROSITC1038',
 'PROSITC1039',
 'PROSITC1040',
 'PROSITC1041',
 'PROSITC1043',
 'PROSITC1044',
 'PROSITC1046',
 'PROSITC1057',
 'PROSITC1061',
 'PROSITC1062',
 'PROSITC1066',
 'PROSITC1070',
 'PROSITC1079',
 'PROSITC1080',
 'PROSITC1082',
 'PROSITC1084',
 'PROSITC1086',
 'PROSITC1089',
 'PROSITC1090',
 'PROSITC1097',
 'PROSITC1100',
 'PROSITC1101',
 'PROSITC1104',
 'PROSITC1109',
 'PROSITC1110',
 'PROSITC1111',
 'PROSITC1119',
 'PROSITC1122',
 'PROSITC1126',
 'PROSITC1128',
 'PROSITC1130',
 'PROSITC1132',
 'PROSITC1135',
 'PROSITC1136',
 'PROSITC1137',
 'PROSITC1140',
 'PROSITC1153',
 'PROSITC1158',
 'PROSITC1174',
 'PROSITC1187',
 'PROSITC1192',
 'PROSITC1193',
 'PROSITC1200',
 'PROSITC1221',
 'PROSITC1222',
 'PROSITC1223',
 'PROSITC1224',
 'PROSITC1227',
 'PROSITC1237',
 'PROSIT

In [46]:
df_temp.to_csv("/csv/finalParticipants.csv", header=True, index=False)